# Design filter mixer design for adc_receiver_v0

In [ ]:
import numpy as np
import scipy
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib widget
#%matplotlib qt

## Parameters of the dec16_fir filter

- 16 samples in //
- 4 taps
- coefficients represented as Q(16, 15)

In [ ]:
n_inputs = 4      # Number of Simultaneous Inputs (2^?)
decimation = 2**n_inputs
TotalTaps = 4     # Total Number of Taps
PFBSize = 5       # We want the same here
alltaps = TotalTaps * 2**PFBSize
fwidth = 1

coef_w = 16
coef_dp = 15

fs = 3700
F_cut = fs/decimation/2

In [ ]:
coeffs = scipy.signal.firwin(alltaps, F_cut , width=None, window='hamming', pass_zero=True, scale=True, nyq=None, fs=fs)
coeffs /= np.max(coeffs)
#windowval = scipy.signal.windows.hamming(alltaps, sym=True)
#coeffs = windowval * np.sinc(fwidth * ((np.arange(alltaps) + 0.5)/(2**PFBSize)-TotalTaps/2))

In [ ]:
def aliased_freq(f_signal, f_sample):
    n = (f_signal / float(f_sample)).round()
    f_alias = abs(f_sample * n - f_signal)
    return f_alias


In [ ]:
def plot_coeff(coeffs, fs):
    plt.figure()
    plt.plot(coeffs)
    plt.show()

    
    w, h = scipy.signal.freqz(coeffs, worN=np.linspace(-fs/2,fs/2,2048), whole=True, fs=fs)
    H = 20*np.log10(abs(h))
    H -= np.max(H)
    
    plt.figure()
    plt.plot(w, H)
    plt.xlim((-fs/decimation*1.2, fs/decimation*1.2))
    plt.ylim((-75, 1))
    
    wa = aliased_freq(w, fs/decimation)
    plt.plot(wa, H)
    plt.plot(-wa, H)
    plt.axvline(-fs/decimation/2, color='r')
    plt.axvline(fs/decimation/2, color='r')
    plt.axvline(-fs/decimation/4, color='k')
    plt.axvline(fs/decimation/4, color='k')
  
    plt.show()

    
    dF0 = fs / decimation
    plt.figure()
    for ch in range(0, 16):
        plt.plot(w+ch*dF0, H, '-')
    
    plt.xlim((-fs/2, fs/2))
    plt.ylim((-75, 1))

    plt.show()


plot_coeff(coeffs, fs)

In [ ]:
coeffs.tofile('dec_fir_coefs.txt', sep=', ')
print('[' + ', '.join( str(c) for c in list(coeffs)) + ']')
print(coeffs.shape)

In [ ]:
def quantize(x, w, pt, mode='round'):
    max_v = (2**(w-1) - 1) / 2**pt
    min_v = (-2**(w-1))    / 2**pt
    print(f"[{min_v},{max_v}]")

    # normalize
    max_x = np.max(x)
    if max_x > max_v:
        x = x / max_x * max_v

    min_x = np.min(x)
    if min_x < min_v:
        x = x / min_x * min_v

    # quantize
    y = x * 2**pt
    if mode == 'round':
        y = np.round(y)
    elif mode == 'trunc':
        y = np.floor(y)
    return y / 2**pt

# x = np.arange(-1024, 1023)/1024
# plt.figure()
# plt.plot(x, quantize(x, 6), '.')
# plt.plot(x, quantize(x, 8), '.')
# plt.plot(x, quantize(x, 10), '.')


In [ ]:
Q = (coef_w, coef_dp)
# Q = (6, 5)
coeffs_q = quantize(coeffs, Q[0], Q[1])
plot_coeff(coeffs_q, fs)


In [ ]:
coeffs_q.tofile(f'dec_fir_coefs_Q{Q[0]}.{Q[1]}.txt', sep=', ')
print('[' + ', '.join( str(c) for c in list(coeffs_q)) + ']')
print(coeffs_q.shape)